In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('../NLP/fake-news/train.csv', error_bad_lines=False)

In [76]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.shape

(20800, 5)

In [6]:
df.dropna(axis=0, how='any', inplace=True)

In [7]:
X=df.drop('label',axis=1)

In [8]:
y=df['label']

In [9]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [78]:
voc_size=13931

In [12]:
#Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages.isna().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [15]:
messages.title[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [16]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [17]:
messages.drop(['id','author','text'],axis=1, inplace=True)

In [18]:
messages

,title
0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired
3,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...
...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [19]:
messages.reset_index(inplace=True)

In [73]:
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
ps = PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
    review=re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [75]:
(corpus[0])

'hous dem aid even see comey letter jason chaffetz tweet'

In [22]:
len(y)

18285

In [25]:
from tensorflow.keras.preprocessing.text import one_hot

In [79]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4055, 6430, 10950, 11436, 9051, 4866, 12884, 1871, 2080, 2709],
 [3308, 10090, 13094, 9670, 11384, 4399, 8874],
 [10592, 8661, 5324, 7281],
 [9781, 6640, 8692, 13343, 1959, 1530],
 [10802, 11384, 3718, 9388, 4403, 2238, 11384, 1570, 7879, 4106],
 [7608,
  8769,
  11918,
  9073,
  12764,
  6695,
  9437,
  1446,
  10177,
  3385,
  10636,
  12860,
  2986,
  5921,
  8874],
 [8368, 6085, 9113, 9651, 4675, 5831, 2035, 6871, 11242, 9637, 11287],
 [8738, 127, 8853, 9570, 906, 9070, 6695, 10222, 11242, 9637, 11287],
 [9616, 3318, 88, 12145, 8522, 8177, 10762, 5241, 6695, 1472],
 [8140, 9690, 11010, 13766, 1295, 1130, 13138, 12194],
 [8973, 7304, 6540, 97, 13093, 12341, 6430, 4228, 3838, 926, 121],
 [13343, 11023, 9051, 8177, 6695, 906],
 [9485, 10415, 2118, 12021, 6297, 9185, 2713, 10664, 7596],
 [10629, 1026, 6263, 12833, 1488, 603, 1895, 11242, 9637, 11287],
 [7026, 9295, 10849, 3199, 7504, 11242, 9637, 11287],
 [6648, 13645, 9025, 1070, 11421, 13313, 11200, 6955, 10790, 10228],
 [1200, 609

In [27]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [80]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[    0     0     0 ...  1871  2080  2709]
 [    0     0     0 ... 11384  4399  8874]
 [    0     0     0 ...  8661  5324  7281]
 ...
 [    0     0     0 ... 11242  9637 11287]
 [    0     0     0 ... 10619  6314  8984]
 [    0     0     0 ... 12677 10610 12981]]


In [81]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            557240    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 613,741
Trainable params: 613,741
Non-trainable params: 0
_________________________________________________________________
None


In [82]:
from sklearn.model_selection import train_test_split
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [83]:
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [84]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=10, batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 4s 361us/sample - loss: 0.3432 - acc: 0.8408 - val_loss: 0.1895 - val_acc: 0.9201
Epoch 2/10
12250/12250 [==============================] - 4s 329us/sample - loss: 0.1237 - acc: 0.9542 - val_loss: 0.1848 - val_acc: 0.9281
Epoch 3/10
12250/12250 [==============================] - 4s 325us/sample - loss: 0.0714 - acc: 0.9743 - val_loss: 0.2035 - val_acc: 0.9233
Epoch 4/10
12250/12250 [==============================] - 4s 323us/sample - loss: 0.0431 - acc: 0.9862 - val_loss: 0.2423 - val_acc: 0.9210
Epoch 5/10
12250/12250 [==============================] - 4s 351us/sample - loss: 0.0286 - acc: 0.9918 - val_loss: 0.2954 - val_acc: 0.9218
Epoch 6/10
12250/12250 [==============================] - 5s 374us/sample - loss: 0.0193 - acc: 0.9938 - val_loss: 0.3739 - val_acc: 0.9173
Epoch 7/10
12250/12250 [==============================] - 4s 341us/sample - loss: 0.0094 - acc: 0.9974 - val_lo

In [39]:
#Performance Metrics and Accuracy

In [85]:
y_pred=model.predict(X_test)

In [86]:
y_pred=np.where(y_pred > 0.5,1,0)

In [87]:
from sklearn.metrics import accuracy_score #Aoc Roc curve
accuracy_score(y_test,y_pred)

0.9181441590720796

In [88]:
from sklearn.metrics import classification_report, confusion_matrix

In [89]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      3419
           1       0.89      0.93      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035



In [90]:
confusion_matrix(y_test,y_pred)

array([[3104,  315],
       [ 179, 2437]], dtype=int64)